In [1]:
import re
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns

In [2]:
TOR_330_dem = pd.read_excel(f'TOR330 Data/5. Clean Data for Data Visualisation/TOR330_dem.xlsx')

In [3]:
checkpoints_bib_df = pd.read_excel(f'TOR330 Data/5. Clean Data for Data Visualisation/TOR330_checkpoints_bibs.xlsx')

In [4]:
all_aid_station_bib_df = pd.read_excel(f'TOR330 Data/5. Clean Data for Data Visualisation/TOR330_all_aid_station_bib_df.xlsx')

In [5]:
lifebase_bib_df = pd.read_excel(f'TOR330 Data/5. Clean Data for Data Visualisation/TOR330_lifebase_bib.xlsx')

In [6]:
# making duration hours 
datasets = [checkpoints_bib_df, lifebase_bib_df, all_aid_station_bib_df,  TOR_330_dem]
for df in datasets:
    df['Duration_hours'] = df['Duration_seconds']/ 3600 

### Who ran too easy or hard at the start?

In [7]:
sub_TOR_330_dem_bib_list = list(TOR_330_dem['PK'][TOR_330_dem['Finish Category'] == 'Sub-130'].unique())

new_bib_list = []
for bib in sub_TOR_330_dem_bib_list:
    df = lifebase_bib_df[
                ((lifebase_bib_df['Duration_hours']> 14) &
                (lifebase_bib_df['Lifebase'] == 'Valgrisenche IN') )
    &
                (lifebase_bib_df['PK']== bib)
               ]
    
    new_bib_list.append(df)
df= pd.concat(new_bib_list)
pk_unique = list(df['PK'].unique())

for pk in pk_unique:
    print(lifebase_bib_df[['PK','Lifebase', 'Timestamp', 'Duration_hours']][(lifebase_bib_df['PK'] == pk)  ], '\n', '*'*40,  '\n',)
    print(TOR_330_dem[['PK', 'Finish Category', 'Duration_hours']][(TOR_330_dem['PK'] == pk)  ], '\n','\n','\n','\n', '*'*40,  '\n',)

                    PK           Lifebase           Timestamp  Duration_hours
1848  TOR330_2024_1058              Start 2024-09-08 12:00:00             NaN
1849  TOR330_2024_1058    Valgrisenche IN 2024-09-09 02:27:01       14.450278
1850  TOR330_2024_1058   Valgrisenche OUT 2024-09-09 03:06:20        0.655278
1851  TOR330_2024_1058           Cogne IN 2024-09-09 19:32:54       16.442778
1852  TOR330_2024_1058          Cogne OUT 2024-09-09 23:01:20        3.473889
1853  TOR330_2024_1058          Donnas IN 2024-09-10 09:59:04       10.962222
1854  TOR330_2024_1058         Donnas OUT 2024-09-10 14:09:33        4.174722
1855  TOR330_2024_1058       Gressoney IN 2024-09-11 09:46:28       19.615278
1856  TOR330_2024_1058      Gressoney OUT 2024-09-11 13:34:21        3.798056
1857  TOR330_2024_1058   Valtournenche IN 2024-09-12 00:12:32       10.636389
1858  TOR330_2024_1058  Valtournenche OUT 2024-09-12 01:43:02        1.508333
1859  TOR330_2024_1058        Ollomont IN 2024-09-12 18:32:40   

                    PK Finish Category  Duration_hours
2582  TOR330_2022_1125         Sub-130      129.061667 
 
 
 
 **************************************** 

                    PK           Lifebase           Timestamp  Duration_hours
47250  TOR330_2021_759              Start 2021-09-12 10:00:00             NaN
47251  TOR330_2021_759    Valgrisenche IN 2021-09-13 00:16:03       14.267500
47252  TOR330_2021_759   Valgrisenche OUT 2021-09-13 01:48:44        1.544722
47253  TOR330_2021_759           Cogne IN 2021-09-13 18:08:00       16.321111
47254  TOR330_2021_759          Cogne OUT 2021-09-13 20:43:05        2.584722
47255  TOR330_2021_759          Donnas IN 2021-09-14 08:09:25       11.438889
47256  TOR330_2021_759         Donnas OUT 2021-09-14 10:30:42        2.354722
47257  TOR330_2021_759       Gressoney IN 2021-09-15 07:14:13       20.725278
47258  TOR330_2021_759      Gressoney OUT 2021-09-15 09:33:12        2.316389
47259  TOR330_2021_759   Valtournenche IN 2021-09-15 19:53:

In [8]:
sub_TOR_330_dem_bib_list = list(TOR_330_dem['PK'][TOR_330_dem['Finish Category'] == 'Sub-140'].unique())

new_bib_list = []
for bib in sub_TOR_330_dem_bib_list:
    df = lifebase_bib_df[
                ((lifebase_bib_df['Duration_hours']< 10) &
                (lifebase_bib_df['Lifebase'] == 'Valgrisenche IN') ) &
                (lifebase_bib_df['PK']== bib)]
    new_bib_list.append(df)
    
df= pd.concat(new_bib_list)

pk_unique = list(df['PK'].unique())

for pk in pk_unique:
    print(lifebase_bib_df[['PK', 'Wave', 'Lifebase', 'Duration_hours']][(lifebase_bib_df['PK'] == pk)  ], '\n', '\n',)
    print(TOR_330_dem[['PK', 'Finish Category', 'Duration_hours']][(TOR_330_dem['PK'] == pk)  ], '\n', '*'*40,  '\n',)

                   PK   Wave           Lifebase  Duration_hours
3360  TOR330_2024_124  Wave1              Start             NaN
3361  TOR330_2024_124  Wave1    Valgrisenche IN        9.784167
3362  TOR330_2024_124  Wave1   Valgrisenche OUT        0.545556
3363  TOR330_2024_124  Wave1           Cogne IN       15.832222
3364  TOR330_2024_124  Wave1          Cogne OUT        1.135000
3365  TOR330_2024_124  Wave1          Donnas IN       14.080556
3366  TOR330_2024_124  Wave1         Donnas OUT        2.827778
3367  TOR330_2024_124  Wave1       Gressoney IN       23.113056
3368  TOR330_2024_124  Wave1      Gressoney OUT        1.560556
3369  TOR330_2024_124  Wave1   Valtournenche IN       13.137222
3370  TOR330_2024_124  Wave1  Valtournenche OUT        1.487778
3371  TOR330_2024_124  Wave1        Ollomont IN       23.116667
3372  TOR330_2024_124  Wave1       Ollomont OUT        1.232778
3373  TOR330_2024_124  Wave1             FINISH       23.335000 
 

                  PK Finish Category

In [9]:
all_aid_station_bib_df[(all_aid_station_bib_df['Bib'] == 1) &
                  (all_aid_station_bib_df['Year'] == 2024) ]

,PK,Year,Race,Bib,Wave,Aid Station,Timestamp,Duration_seconds,Duration_hours
26650,TOR330_2024_1,2024,TOR330,1,Wave1,Start,2024-09-08 10:00:00,NaN,NaN
26651,TOR330_2024_1,2024,TOR330,1,Wave1,Baite Youlaz,NaT,NaN,NaN
26652,TOR330_2024_1,2024,TOR330,1,Wave1,La Thuile,NaT,NaN,NaN
26653,TOR330_2024_1,2024,TOR330,1,Wave1,Rifugio Deffeyes,NaT,NaN,NaN
26654,TOR330_2024_1,2024,TOR330,1,Wave1,Planaval,NaT,NaN,NaN
26655,TOR330_2024_1,2024,TOR330,1,Wave1,Valgrisenche IN,NaT,NaN,NaN
26656,TOR330_2024_1,2024,TOR330,1,Wave1,Valgrisenche OUT,NaT,NaN,NaN
26657,TOR330_2024_1,2024,TOR330,1,Wave1,Chalet Epee,NaT,NaN,NaN
26658,TOR330_2024_1,2024,TOR330,1,Wave1,Rhemes-Notre-Dame,NaT,NaN,NaN
26659,TOR330_2024_1,2024,TOR330,1,Wave1,Eaux Rousse,NaT,NaN,NaN


### Examining Colle Franco

In [10]:
# reading in Raw Data
races = ['TOR330']
years = [ 
#     '2021',
#         '2022',
#          '2023', 
    '2024'
        ]

TORX_df = {}

for race in races:
    for year in years:
        df = pd.read_excel(f'{race} Data/1. 100x100trail/{race}_{year}.xlsx',
                                 dtype={'Start Date': 'string',
                                        'Year': 'string'})
        print(f'{race}_{year} {df.shape}')
        # Store the DataFrame in the dictionary with a key like 'TOR330_2021'
        TORX_df[f'{race}_{year}'] = df
    print('*'*50)
    
TORX_df_concat = pd.concat(TORX_df)
TOR330 = TORX_df_concat[TORX_df_concat['Year'] == year]

TOR330_2024 (39181, 10)
**************************************************


In [11]:
TOR330[(TOR330['Bib'] == 1) ]

Name  Bib Sex Nationality    Race  Year Category  \
TOR330_2024 28063  Colle Franco    1   M          IT  TOR330  2024       V1   
            28064  Colle Franco    1   M          IT  TOR330  2024       V1   
            28065  Colle Franco    1   M          IT  TOR330  2024       V1   

                                            Place                       Time  \
TOR330_2024 28063  640 - Pas Entre Deux Sauts 450  2024-09-14T07:58:11+02:00   
            28064            560 - Bruson Arp 130  2024-09-11T20:22:22+02:00   
            28065                     010 - START  2024-08-24T23:40:09+02:00   

                   Status  
TOR330_2024 28063   False  
            28064   False  
            28065   False

In [12]:
all_aid_station_bib_df[all_aid_station_bib_df['Bib'] ==  1]

,PK,Year,Race,Bib,Wave,Aid Station,Timestamp,Duration_seconds,Duration_hours
26650,TOR330_2024_1,2024,TOR330,1,Wave1,Start,2024-09-08 10:00:00,NaN,NaN
26651,TOR330_2024_1,2024,TOR330,1,Wave1,Baite Youlaz,NaT,NaN,NaN
26652,TOR330_2024_1,2024,TOR330,1,Wave1,La Thuile,NaT,NaN,NaN
26653,TOR330_2024_1,2024,TOR330,1,Wave1,Rifugio Deffeyes,NaT,NaN,NaN
26654,TOR330_2024_1,2024,TOR330,1,Wave1,Planaval,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...
163095,TOR330_2021_1,2021,TOR330,1,No Wave in 2021,Bosses,2021-09-14 23:19:05,4805.0,1.334722
163096,TOR330_2021_1,2021,TOR330,1,No Wave in 2021,Rifugio Frassati,2021-09-15 01:19:06,7201.0,2.000278
163097,TOR330_2021_1,2021,TOR330,1,No Wave in 2021,Pas Entre Deux Sauts,NaT,NaN,NaN
163098,TOR330_2021_1,2021,TOR330,1,No Wave in 2021,Monte de la Saxe,2021-09-15 04:04:48,NaN,NaN


In [13]:
sub_TOR_330_dem_bib_list = list(TOR_330_dem['PK'][TOR_330_dem['Finish Category'] == 'Sub-90'].unique())

new_bib_list = []
for bib in sub_TOR_330_dem_bib_list:
    df = lifebase_bib_df[
                ((lifebase_bib_df['Duration_hours']> 8) &
                (lifebase_bib_df['Lifebase'] == 'Valgrisenche OUT') ) &
                (lifebase_bib_df['PK']== bib)]
    new_bib_list.append(df)
    
df= pd.concat(new_bib_list)

pk_unique = list(df['PK'].unique())

for pk in pk_unique:
    print(lifebase_bib_df[['PK', 'Wave', 'Lifebase', 'Duration_hours']][(lifebase_bib_df['PK'] == pk)  ], '\n', '\n',)
    print(TOR_330_dem[['PK', 'Finish Category', 'Duration_hours']][(TOR_330_dem['PK'] == pk)  ], '\n', '*'*40,  '\n',)

In [14]:
TOR_330_dem['Wave'][TOR_330_dem ['Year'] == 2021].unique()

array(['No Wave in 2021'], dtype=object)

### Getting Average and Median time for lifebases

In [23]:
def creating_time_stats(df, column, category_order):
    df_merge = df.merge(
        TOR_330_dem[['PK', 'Finish Category']],
        on=['PK'],
        how='left')


    df_merge['Duration_seconds'][df_merge['Duration_seconds']  <= 0] = np.nan

    stats_df = df_merge.groupby(['Finish Category', column])['Duration_seconds'].describe().reset_index(drop = False)


    stats_df[[ 'mean', 'std', 'min', '25%',
           '50%', '75%', 'max']] = stats_df[[ 'mean', 'std', 'min', '25%',
           '50%', '75%', 'max']].round(0)

    # Set 'Finish Category' as a categorical column with the defined order
    stats_df[column] = pd.Categorical(
        stats_df[column],
        categories=category_order,
        ordered=True)
    
    stats_df = stats_df.sort_values(by=column, ascending = True)
    
    

    for finish_category in df_merge['Finish Category'].unique():
        print(finish_category)
        stats_df.loc[
                (stats_df['Finish Category'] == finish_category), 'running_total_mean_seconds'
            ] =     stats_df.loc[
                (stats_df['Finish Category'] == finish_category), 'mean'
            ].cumsum()


        stats_df.loc[
                (stats_df['Finish Category'] == finish_category), 'running_total_median_seconds'
            ] =     stats_df.loc[
                (stats_df['Finish Category'] == finish_category), '50%'
            ].cumsum()


    stats_df = stats_df[['Finish Category', column,
            'count', 'mean', '50%', 'std', 'min', 'max', 
            'running_total_mean_seconds', 'running_total_median_seconds']]


    stats_df = stats_df.rename(columns={'count': f'Count_Finish_Category_{column}_seconds',
                                      'mean': f'Mean_Finish_Category_{column}_seconds',
                                      'std': f'STD_Finish_Category{column}t_seconds',
                                      '50%': f'Median_Finish_Category_{column}_seconds',
                                      'min': f'Min_Finish_Category_{column}_seconds', 
                                      'max': f'Max_Finish_Category_{column}_seconds'})
    
    stats_df.to_excel(f'{race} Data/5. Clean Data for Data Visualisation/{race}_{column}_stats_df.xlsx', index = False)

#     print(stats_df[stats_df['Finish Category'] == 'Sub-130'])
    return stats_df

In [21]:
lifebase_stats_df = creating_time_stats(df = lifebase_bib_df, 
                    column = 'Lifebase', 
                    category_order = ['Start',
                            'Valgrisenche IN','Valgrisenche OUT',
                            'Cogne IN',  'Cogne OUT',
                            'Donnas IN', 'Donnas OUT', 
                            'Gressoney IN','Gressoney OUT',
                            'Valtournenche IN','Valtournenche OUT', 
                            'Ollomont IN','Ollomont OUT',
                            'FINISH'])

lifebase_stats_df

C:\Users\Karina\AppData\Local\Temp\ipykernel_21476\100497761.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge['Duration_seconds'][df_merge['Duration_seconds']  <= 0] = np.nan


Sub-70
Sub-80
Sub-90
Sub-100
Sub-110
Sub-120
Sub-130
Sub-140
Sub-150
Over-150
DNF
nan
   Finish Category           Lifebase  Count_Finish_Category_Lifebase_seconds  \
80         Sub-130    Valgrisenche IN                                   492.0   
81         Sub-130   Valgrisenche OUT                                   486.0   
71         Sub-130           Cogne IN                                   481.0   
72         Sub-130          Cogne OUT                                   463.0   
73         Sub-130          Donnas IN                                   466.0   
74         Sub-130         Donnas OUT                                   487.0   
76         Sub-130       Gressoney IN                                   488.0   
77         Sub-130      Gressoney OUT                                   491.0   
82         Sub-130   Valtournenche IN                                   491.0   
83         Sub-130  Valtournenche OUT                                   491.0   
78         Sub-130     

,Finish Category,Lifebase,Count_Finish_Category_Lifebase_seconds,Mean_Finish_Category_Lifebase_seconds,Median_Finish_Category_Lifebase_seconds,STD_Finish_CategoryLifebaset_seconds,Min_Finish_Category_Lifebase_seconds,Max_Finish_Category_Lifebase_seconds,running_total_mean_seconds,running_total_median_seconds
108,Sub-150,Valgrisenche IN,610.0,48434.0,48110.0,5167.0,36183.0,65097.0,48434.0,48110.0
38,Sub-100,Valgrisenche IN,80.0,32851.0,32880.0,2535.0,25495.0,40300.0,32851.0,32880.0
122,Sub-70,Valgrisenche IN,5.0,25546.0,25476.0,353.0,25288.0,26146.0,25546.0,25476.0
52,Sub-110,Valgrisenche IN,175.0,35626.0,35430.0,3381.0,28009.0,47642.0,35626.0,35430.0
80,Sub-130,Valgrisenche IN,492.0,41154.0,40887.0,4248.0,30261.0,55736.0,41154.0,40887.0
...,...,...,...,...,...,...,...,...,...,...
84,Sub-140,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,Sub-150,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,Sub-70,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,Sub-80,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
checkpoints_stats_df = creating_time_stats(df = checkpoints_bib_df, 
                    column = 'Checkpoint', 
                    category_order = ['Start', 'La Thuile', 'Valgrisenche IN', 'Valgrisenche OUT',
                                       'Eaux Rousse', 'Cogne IN', 'Cogne OUT', 'Donnas IN', 'Donnas OUT',
                                       'Rifugio della Barma', 'Niel La Gruba', 'Gressoney IN',
                                       'Gressoney OUT', 'Champoluc', 'Valtournenche IN',
                                       'Valtournenche OUT', 'Oyace', 'Ollomont IN', 'Ollomont OUT',
                                       'FINISH'])

C:\Users\Karina\AppData\Local\Temp\ipykernel_21476\100497761.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge['Duration_seconds'][df_merge['Duration_seconds']  <= 0] = np.nan


Sub-70
Sub-80
Sub-90
Sub-100
Sub-110
Sub-120
Sub-130
Sub-140
Sub-150
Over-150
DNF
nan
    Finish Category           Checkpoint  \
115         Sub-130                Start   
109         Sub-130            La Thuile   
116         Sub-130      Valgrisenche IN   
117         Sub-130     Valgrisenche OUT   
105         Sub-130          Eaux Rousse   
101         Sub-130             Cogne IN   
102         Sub-130            Cogne OUT   
103         Sub-130            Donnas IN   
104         Sub-130           Donnas OUT   
114         Sub-130  Rifugio della Barma   
110         Sub-130        Niel La Gruba   
107         Sub-130         Gressoney IN   
108         Sub-130        Gressoney OUT   
100         Sub-130            Champoluc   
118         Sub-130     Valtournenche IN   
119         Sub-130    Valtournenche OUT   
113         Sub-130                Oyace   
111         Sub-130          Ollomont IN   
112         Sub-130         Ollomont OUT   
106         Sub-130               

In [24]:
all_aid_station_bib_stats_df = creating_time_stats(df = all_aid_station_bib_df, 
                    column = 'Aid Station', 
                    category_order = ['Start',
                                    'Baite Youlaz', 'La Thuile', 'Rifugio Deffeyes',
                                   'Planaval', 'Valgrisenche IN', 'Valgrisenche OUT', 'Chalet Epee',
                                   'Rhemes-Notre-Dame', 'Eaux Rousse', 'Rifugio Sella', 'Cogne IN',
                                   'Cogne OUT', 'Goilles', 'Rifugio Dondena', 'Chardonney', 'Pontboset',
                                   'Donnas IN', 'Donnas OUT', 'Perloz', 'Sassa', 'Rifugio Coda',
                                   'Rifugio della Barma', 'Lago Chiaro', 'Col della Vecchia',
                                   'Niel La Gruba', 'Loo', 'Gressoney IN', 'Gressoney OUT',
                                   'Rifugio Alpenzu', 'Champoluc', 'Rifugio Grand Tournalin',
                                   'Valtournenche IN', 'Valtournenche OUT', 'Rifugio Barmasse', 'Vareton',
                                   'Rifugio Magià', 'Rifugio Cuney', 'Bivacco R. Clermont', 'Oyace',
                                   'Bruson Arp', 'Ollomont IN', 'Ollomont OUT', 'Rifugio Champillon',
                                   'Ponteille Desot', 'Bosses', 'Rifugio Frassati', 'Pas Entre Deux Sauts',
                                   'Monte de la Saxe', 'FINISH'])

all_aid_station_bib_stats_df

C:\Users\Karina\AppData\Local\Temp\ipykernel_21476\1535442705.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge['Duration_seconds'][df_merge['Duration_seconds']  <= 0] = np.nan


Sub-70
Sub-80
Sub-90
Sub-100
Sub-110
Sub-120
Sub-130
Sub-140
Sub-150
Over-150
DNF
nan


,Finish Category,Aid Station,Count_Finish_Category_Aid Station_seconds,Mean_Finish_Category_Aid Station_seconds,Median_Finish_Category_Aid Station_seconds,STD_Finish_CategoryAid Stationt_seconds,Min_Finish_Category_Aid Station_seconds,Max_Finish_Category_Aid Station_seconds,running_total_mean_seconds,running_total_median_seconds
244,Sub-120,Start,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,Over-150,Start,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
344,Sub-140,Start,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
294,Sub-130,Start,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
444,Sub-70,Start,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
363,Sub-150,FINISH,607.0,4977.0,4800.0,2087.0,2322.0,41505.0,525230.0,508938.0
13,DNF,FINISH,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
513,Sub-90,FINISH,43.0,3170.0,3130.0,673.0,1876.0,4552.0,309385.0,300283.0
163,Sub-110,FINISH,174.0,3458.0,3338.0,861.0,1632.0,7922.0,381744.0,368506.0


In [48]:
bib_df['Lifebase'].unique()

array([' Start', 'Valgrisenche IN', 'Valgrisenche OUT', 'Cogne IN',
       'Cogne OUT', 'Donnas IN', 'Donnas OUT', 'Gressoney IN',
       'Gressoney OUT', 'Valtournenche IN', 'Valtournenche OUT',
       'Ollomont IN', 'Ollomont OUT', 'FINISH'], dtype=object)

### Easy Finishers Running Total

In [52]:
# Filter the dataframe for the current unique_bib
bib_df = lifebase_bib_df[lifebase_bib_df['PK'] == 'TOR330_2023_1440'].reset_index(drop = True)

# bib_df['Duration'] = bib_df['Timestamp'].diff()

bib_start_date = bib_df['Timestamp'][bib_df['Lifebase'] == " Start"].values[0]
print(bib_start_date)
print(type(bib_start_date))


for idx, row in bib_df.iterrows():
    bib_df.at[idx, 'Running Total'] = bib_df.at[idx, 'Timestamp']- bib_start_date
bib_df

2023-09-10T12:00:00.000000000
<class 'numpy.datetime64'>


,PK,Year,Race,Bib,Wave,Lifebase,Timestamp,Duration_seconds,Duration_hours,Running Total
0,TOR330_2023_1440,2023,TOR330,1440,Wave2,Start,2023-09-10 12:00:00,NaN,NaN,0 days 00:00:00
1,TOR330_2023_1440,2023,TOR330,1440,Wave2,Valgrisenche IN,2023-09-11 02:24:30,51870.0,14.408333,0 days 14:24:30
2,TOR330_2023_1440,2023,TOR330,1440,Wave2,Valgrisenche OUT,2023-09-11 04:21:59,7049.0,1.958056,0 days 16:21:59
3,TOR330_2023_1440,2023,TOR330,1440,Wave2,Cogne IN,2023-09-12 02:20:54,79135.0,21.981944,1 days 14:20:54
4,TOR330_2023_1440,2023,TOR330,1440,Wave2,Cogne OUT,2023-09-12 02:59:16,2302.0,0.639444,1 days 14:59:16
5,TOR330_2023_1440,2023,TOR330,1440,Wave2,Donnas IN,2023-09-12 17:53:05,53629.0,14.896944,2 days 05:53:05
6,TOR330_2023_1440,2023,TOR330,1440,Wave2,Donnas OUT,2023-09-12 19:43:39,6634.0,1.842778,2 days 07:43:39
7,TOR330_2023_1440,2023,TOR330,1440,Wave2,Gressoney IN,2023-09-13 20:45:10,90091.0,25.025278,3 days 08:45:10
8,TOR330_2023_1440,2023,TOR330,1440,Wave2,Gressoney OUT,2023-09-13 23:37:01,10311.0,2.864167,3 days 11:37:01
9,TOR330_2023_1440,2023,TOR330,1440,Wave2,Valtournenche IN,2023-09-14 13:58:04,51663.0,14.350833,4 days 01:58:04


In [55]:
# Filter the dataframe for the current unique_bib
bib_df = lifebase_bib_df[lifebase_bib_df['PK'] == 'TOR330_2023_532'].reset_index(drop = True)

bib_start_date = bib_df['Timestamp'][bib_df['Lifebase'] == " Start"].values[0]

for idx, row in bib_df.iterrows():
    bib_df.at[idx, 'Running Total Duration'] = bib_df.at[idx, 'Timestamp']- bib_start_date
    
    bib_df['Running Total Duration_seconds'] = pd.to_timedelta(
        bib_df['Running Total Duration'], errors='coerce').dt.total_seconds()


    
    
    
bib_df

,PK,Year,Race,Bib,Wave,Lifebase,Timestamp,Duration_seconds,Duration_hours,Running Total Duration,Running Total Duration_seconds
0,TOR330_2023_532,2023,TOR330,532,Wave1,Start,2023-09-10 10:00:00,NaN,NaN,0 days 00:00:00,0.0
1,TOR330_2023_532,2023,TOR330,532,Wave1,Valgrisenche IN,2023-09-11 00:40:41,52841.0,14.678056,0 days 14:40:41,52841.0
2,TOR330_2023_532,2023,TOR330,532,Wave1,Valgrisenche OUT,2023-09-11 05:07:26,16005.0,4.445833,0 days 19:07:26,68846.0
3,TOR330_2023_532,2023,TOR330,532,Wave1,Cogne IN,2023-09-12 01:17:11,72585.0,20.162500,1 days 15:17:11,141431.0
4,TOR330_2023_532,2023,TOR330,532,Wave1,Cogne OUT,2023-09-12 04:10:22,10391.0,2.886389,1 days 18:10:22,151822.0
5,TOR330_2023_532,2023,TOR330,532,Wave1,Donnas IN,2023-09-12 19:12:23,54121.0,15.033611,2 days 09:12:23,205943.0
6,TOR330_2023_532,2023,TOR330,532,Wave1,Donnas OUT,2023-09-12 20:06:33,3250.0,0.902778,2 days 10:06:33,209193.0
7,TOR330_2023_532,2023,TOR330,532,Wave1,Gressoney IN,2023-09-13 23:36:16,98983.0,27.495278,3 days 13:36:16,308176.0
8,TOR330_2023_532,2023,TOR330,532,Wave1,Gressoney OUT,NaT,NaN,NaN,NaT,NaN
9,TOR330_2023_532,2023,TOR330,532,Wave1,Valtournenche IN,NaT,NaN,NaN,NaT,NaN
